# How to run

You can run each cell in order with shift+enter, or run the whole thing at once by using the Kernel menu up above and hitting 'Restart & Run All'

This notebook expects a folder structure as follows
```
---
 | powers.ipynb
 |-data\Orcus - Powers.xlsx
 |-output
```

and will take data\Orcus - Powers.xlsx and create a markdown file output\powers.md

We use Pandas to read in the excel spreadsheet, and define a function which takes a row in the Pandas DataFrame and puts all the values in the right place (with a little logic to handle fields which aren't always there) to create a markdown formatted string.

It's pretty similar to what the existing Mail Merge does, but we have complete control over how we build the string and map values. The key Pythonic bit is the curly braces {} and the `**dict`. Curly braces indicate a place that we want to input a value in a string with a **key**, and ** takes a dictionary and puts the values with the proper keys in the proper place.  

```
'{hello}'.format(**{'hello':'world', 'not_needed':'mars'})
evaluates to
'world'
```


Then we join that list of strings and save a file.  For more on Python, I recommend [Automate the Boring Stuff with Python](https://automatetheboringstuff.com/)

In [1]:
# cell 0, standard imports

import pandas as pd
import os

dot  = '●'

In [2]:
#cell 1, read in data from filename

filename = 'Orcus - Powers.xlsx'

#by default Pandas loads missing values with ugly NaNs, so we replace those with the empty string ''
data = pd.read_excel(os.path.join('data', filename)).fillna('')
data.head()

,Source,List,Name,Flavor,Category,Frequency,Tier,Tags,Action,Range,...,Miss,Effect,Special,Boost,Key Ability,Secondary Ability,Tradition,Details,Maintain Action,Maintain Text
0,Advanced Combat,Advanced Combat,Chapter,,,,0,,,,...,,,,,,,,,,
1,Advanced Combat,Advanced Combat,Damage Power Source,,Attack,At-Will,,"Martial, Weapon",Standard,Melee or Ranged,...,,,,,,,,,,
2,Advanced Combat,Advanced Combat,Disarm,,Attack,At-Will,,"Martial, Weapon",Standard,Melee,...,,You provoke an attack of opportunity from the ...,A disarmed creature cannot use powers with the...,,,,,,,
3,Advanced Combat,Advanced Combat,Ignite,,Attack,At-Will,,"Martial, Weapon",Standard,Melee or Ranged,...,,,You must have a source of fire to use this power.,,,,,,,
4,Advanced Combat,Advanced Combat,Lure,,Attack,At-Will,,Focus,Standard,Near,...,,The target is immune to the *lure* power until...,Depending on the susceptibility of the creatur...,,,,,,,


In [3]:
#cell 2, examine 5 random powers

data.sample(5)

,Source,List,Name,Flavor,Category,Frequency,Tier,Tags,Action,Range,...,Miss,Effect,Special,Boost,Key Ability,Secondary Ability,Tradition,Details,Maintain Action,Maintain Text
662,Epic Path,Most Dangerous,Chapter,,,,0,,,,...,,,,,,,,,,
274,Discipline,Frontline Fighting,Smack-Attack,Your opponent rushes you. You direct their en...,Utility,Daily,6,Martial,Counter,Self,...,,The attack misses. Shift to any square adjacen...,,,,,,,,
765,Combat,Combat,Basic Ranged Attack,,Attack,At-Will,Feature,Weapon,Standard,Ranged,...,,,,,,,,,,
299,Discipline,Golden Lion,Pack Pounce,By capitalizing on the opportunities granted b...,Attack,At-Will,1,"Martial, Weapon",Standard,Melee or Ranged,...,,,,,,,,,,
142,Discipline,Born Leadership,Lead the Group,You stand in front as a proud example. Where ...,Attack,Daily,15,"Martial, Weapon",Swift,Near,...,,"Until the end of the encounter, on the target’...",,,,,,,,


In [4]:
#cell 3
#we define a function which takes in a row of this power dataframe and returns an md string

def power_to_md(row):
    
    #converting the row to a dictionary let's use do some cool stuff with placing values in strings using .format(**row_dict)
    row_dict = dict(row)
    
    #remove leading and trailing whitespace
    for k, v in row_dict.items():
        row_dict[k] = str(v).strip()
    
    # we use completion to check a few edge cases
    completion = dict()
    for key,value in row.items():
        if value:
            completion[key] = value
    
    if row_dict.get('Name') == 'Chapter':
        md = '## {List}  '.format(**row_dict)
        if row['Key Ability']:
            md += "\n**Key Ability:** {Key Ability}  ".format(**row_dict)
        if row['Secondary Ability']:
            md += "\n**Secondary Ability:** {Secondary Ability}  ".format(**row_dict)
        if row['Details']:
            md += "\n\n{Details}  ".format(**row_dict)
        return md
    
   
    # we check if nothing is filling in and return an empty row
    if list(completion.keys()) == []:
        md = ''
        return md
    
    #start with name. md is the string we'll be outputing
    md = "> #### {Name}  ".format(**row_dict)
          
    if row['Flavor']:
        md += '\n> *{Flavor}*  '.format(**row_dict)
        
    #modify tags a little, since they aren't always present    
    if row_dict.get('Tags'):
        row_dict['Tags'] = '● **{Tags}**'.format(**row_dict)
    
    #this is a complex chunk
    md += """\n> **{Frequency}** **{Category}** **{Tier}** (**{Action} Action**) {Tags}  """.format(**row_dict)
    
    if row['Range']:
        md += '\n> **{Range}** {Range Details}  '.format(**row_dict)
    
    #if a field exists, we add it
    if row['Requirement']:
        md += '\n> **Requirements** {Requirement}  '.format(**row_dict)
        
    if row['Trigger']:
        md += '\n> **Trigger** {Trigger}  '.format(**row_dict)
        
    if row['Attack']:
        md += '\n> **Attack** {Attack} vs {Defense}  '.format(**row_dict)
        
    if row['Hit']:
        md += '\n> **Hit** {Hit}  '.format(**row_dict)
    
    if row['Miss']:
        md += '\n> **Miss** {Miss}  '.format(**row_dict)
        
    if row['Effect']:
        md += '\n> **Effect** {Effect}  '.format(**row_dict)
        
    if row['Special']:
        md += '\n> **Special** {Special}  '.format(**row_dict)
        
    if row['Maintain Action']:
        md += '\n> **Maintain {Maintain Action}** {Maintain Text}  '.format(**row_dict)
        
    if row['Boost']:
        md += '\n> **Boost** {Boost}  '.format(**row_dict)
    
    #finally we remove the empty tags
    return md.replace('****','')


In [5]:
#cell 4
#this cell converts the dataframe into a list of markdown formatted strings using the function defined above

output = []

for idx, row in data.iterrows():
    md = power_to_md(row)
    if md: #this removes empty lines, which you may not want
        output.append(md)

In [6]:
#cell 5
#we can look at elements here using Python list slicing

for elem in output[100:110]:
    print(elem)
    print('')

> #### An Unwise Action  
> *Enemies train their weapons.  You advise your opponents of the wisdom of such a risk, and then show them why.*  
> **Daily** **Attack** **19** (**Move Action**) ● **Martial, Weapon**  
> **Near** burst 1, all enemies you can see  
> **Effect** The target drops any weapons it is holding, and you make a basic attack against it.  
> **Special** You can choose to grab any weapon a target drops and use it for subsequent basic attack/s. You can drop a weapon you pick up as part of this power as a free action.  

> #### Mission Objective  
> *While everyone attacks, you make a move to the goal.*  
> **Encounter** **Attack** **23** (**Free Action**) ● **Martial, Weapon**  
> **Self**   
> **Trigger** You hit an enemy.  
> **Effect** The target takes 2dW extra damage from the triggering attack, and it cannot attack you until the start of your next turn unless that attack includes at least one other creature.  

> #### Attack Deflection  
> *You maneuver in such a wa

In [7]:
#cell 6
# now we have to save to disk

fname =  'Orcus Powers.md'

with open(os.path.join('output', fname), 'w', encoding='utf-8') as f:
    f.write('\n\n'.join(output))